## MORL Setup

### Initialize all metrics that will be used in the reward model

In [ ]:
import sys
sys.path.append('metrics/SemanticSimilarity/')
sys.path.append('metrics/BARTScore/')

sent = ['When applying the Naturalistic Driving Film in the design process, there are several aspects that need to be taken into consideration.']
ref = ['When apply Naturalistic Driving Film into the design process, there are several aspects need to take into consideration.',]

import torch

device = 0 if torch.cuda.is_available() else "cpu"

### Initialize Semantic Similarity (SIM)
Download needed model files: sim.pt and sim.sp.30k.model from https://github.com/martiansideofthemoon/style-transfer-paraphrase/tree/master/style_paraphrase/evaluation. 
And modify the corresponding pathes in test_sim.py.

In [ ]:
from test_sim import find_similarity

#test find_similarity
sim_score = find_similarity(sent,ref)

print(sim_score)

### Initialize Perplexity (PPL)
Tune your own GPT2-Large model and put the tuned model under [metrics/Perplexity/Model-for-PPL](metrics/Perplexity/Model-for-PPL)

In [ ]:
import evaluate
perplexity = evaluate.load("perplexity", module_type="metric",cache_dir = None)
LM_path = "metrics/Perplexity/Model-for-PPL"

input_texts = sent

#test PPL calculation
results = perplexity.compute(model_id=LM_path,
                             add_start_token=False,
                             predictions=input_texts)

print(list(results.keys()))
print(round(results["mean_perplexity"], 2))
print(round(results["perplexities"][0], 2))

### Initialize BartScorer (BARTS)
Download needed model file: bart_score.pth from https://github.com/neulab/BARTScore and put it under [metrics/BARTScore/Model-for-BARTS/](.metrics/BARTScore/Model-for-BARTS/)

In [ ]:
from bart_score import BARTScorer

#test BARTScorer
bart_scorer = BARTScorer(device = "cuda:0" if device == 0 else "cpu", checkpoint='facebook/bart-large-cnn')
bart_scorer.load(path='metrics/BARTScore/Model-for-BARTS/bart_score.pth')
bart_score = bart_scorer.score(sent,ref)

print(bart_score)

### Initialize Transfer Accuracy Classifier (ACC)
Train your own RoBERTa-Large classifier fine-tuned on the CoLA or AESW2016 dataset following huggingface tutorial and put it under metrics/TransferAccuracy/Model-for-ACC/

In [ ]:
from transformers import pipeline, AutoTokenizer

sentiment_pipe = pipeline("text-classification", model="metrics/TransferAccuracy/Model-for-ACC", device= device, tokenizer = AutoTokenizer.from_pretrained('roberta-large', max_length = 256))

### PPO Configuration

In [ ]:
from tqdm import tqdm
import pandas as pd
tqdm.pandas()

from datasets import load_dataset

from trl import PPOTrainer, PPOConfig

In [ ]:
config = PPOConfig(
    model_name="path to your pretrained bart model", #path to your pretrained bart-large model for seq2seq text generation
    learning_rate=1e-6,
    #log_with="wandb",
    batch_size=32
)

sent_kwargs = {
    "return_all_scores": True,
    "function_to_apply": "none",
    "batch_size": 16
}

### Load AWF data

In [ ]:
def build_awf_dataset(config, dataset_path):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should 
    customize this function to train the model on its own dataset.
    
    Args:
        dataset_name (`str`): 
            The name of the dataset to be loaded.
    
    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """

    tokenizer = AutoTokenizer.from_pretrained(config.model_name)

    ds = load_dataset('csv',data_files=dataset_path)['train']
    ds = ds.rename_columns({'text': 'review'})

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"][:-1])
        sample["query"] = tokenizer.decode(sample["input_ids"],skip_special_tokens=True)
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type='torch')
    return ds

In [ ]:
dataset = build_awf_dataset(config,'AWF-dataset/dev.0.csv')

In [ ]:
dataset

In [ ]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

### Load your pre-trained BART-Large language model

In [ ]:
from trl import AutoModelForSeq2SeqLMWithValueHead

model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

### Initialize PPOTrainer
The `PPOTrainer` takes care of device placement and optimization later on:

In [ ]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

In [ ]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
   device = 0 if torch.cuda.is_available() else "cpu" # to avoid a `pipeline` bug

### Generation settings

In [ ]:
gen_kwargs = {
    "min_length":-1,
    "max_length":256,
    "top_k": 0.0,
    #"top_p": 1.0,
    'do_sample': True,
    'early_stopping' : False, 
    'num_beams' : 4, 
    #'no_repeat_ngram_size': 2
}

## Optimize model

### Training loop

The training loop consists of the following main steps:
1. Get the generated refined paragraph from the policy network (BART-Large)
2. Get the scores of generated paragraph on each chosen metrics
3. Get the final reward as a weighted sum of all scores got in Step 2
4. Optimize policy with PPO using the (query, response, reward) triplet. Here, query is the input paragraphs

In [ ]:
from math import exp
import numpy as np

weights = [0.01,1.0,0.01,0.01] #weights for bart-score, -PPL/200, similarity, classification_pipeline

for ep in range(5):
    for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
        query_tensors = batch['input_ids']

        response_tensors = []
        for query in query_tensors:
            response = ppo_trainer.generate(query, **gen_kwargs)
            response_tensors.append(response.squeeze())
        batch['response'] = [tokenizer.decode(r.squeeze(),skip_special_tokens=True) for r in response_tensors]

        texts = batch['response']
        references = batch['review']
        
        #get BART Score
        BS_rewards = [torch.tensor(bart_scorer.score([reference],[output])[0]) for output, reference in zip(texts,references)]
        
        #get Perplexity Score
        PPL_rewards = []
        for output in texts:
            try:
                PPL_rewards.append(torch.tensor(-perplexity.compute(model_id=LM_path,
                                                   add_start_token=True, 
                                                   predictions=[output])["mean_perplexity"]/200.0))
            except:
                print("""Warning: Model generated a paragraph contains less than 2 tokens. If this warning shows frequently, 
                      the leaning process is probably crushed. Please try to restart this notebook and tune the model again.""")
                PPL_rewards.append(torch.tensor([-10000]))
        
        #get Semantic Similarity Score
        SIM_rewards = [torch.tensor(find_similarity([output],[reference])[0]) for output, reference in zip(texts,references)]
        
        pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
        AF_rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]
        #sometimes the below implementation of AF_rewards is better
        #AF_rewards = [torch.tensor(exp(output[1]["score"])/(exp(output[1]["score"])+exp(output[0]["score"]))) for output in pipe_outputs]

        rewards = [weights[0]*a + weights[1]*b + weights[2]*c + weights[3]*d for a,b,c,d in zip(BS_rewards,PPL_rewards,SIM_rewards,AF_rewards)]
        
        #### Run PPO step 
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
        ppo_trainer.log_stats(stats, batch, rewards)

    print('epoch : ', ep+1)
    
    #get the generation result on test set after each epoch, comment all lines below to accelerate training (not recommand, the last epoch is unlikely the best)
    
    with open('AWF-dataset/paragraph_native_test.0') as inp, open('tmp/model_output.txt','w') as out:
        lines = inp.readlines()
        for sent in lines:
            if sent.endswith('/n'):
                sent = sent[:-1]
            sent_encoded = tokenizer.encode(sent,return_tensors='pt').cuda()
            response = model.generate(sent_encoded, **gen_kwargs)
            gen_sent = tokenizer.decode(response.squeeze(),skip_special_tokens=True)
            out.write(gen_sent+'\n')

    lines = []
    
    with open('tmp/model_output.txt') as inp, open('tmp/model_output_processed.txt','w',encoding='utf-8') as out:
        lines = inp.readlines()
        #print(len(lines))
        count = 0
        for line in lines:
            if line.endswith('\n'):
                line = line[:-1]
            if line.rfind('.') != -1:
                out.write(line[:line.rfind('.')+1]+'\n')
            else:
                out.write(line+'\n')

    with open('tmp/model_output.txt') as inp1, open('tmp/model_output_processed.txt') as inp2:
        with open('tmp/model_output-'+str(ep+1)+'.txt','w',encoding='utf-8') as out1, open('tmp/model_output_processed-'+str(ep+1)+'.txt','w',encoding='utf-8') as out2:
            out1.write(inp1.read())
            out2.write(inp2.read())
    
    #evaluate and save model after each epoch, comment all lines below to accelerate training (not recommand, the last epoch is unlikely the best)

    #AF score
    print('AF score:')
    results = []
    with open('tmp/model_output.txt') as inp:
        preds = sentiment_pipe (inp.readlines())
        for pred in preds:
            if pred['label'] == 'LABEL_0':
                results.append(0)
            else:
                results.append(1)

    AF_1 = round(np.mean(results)*100,2)

    print('ACC of raw-output: ' + str(AF_1) + '%.')

    results = []
    with open('tmp/model_output_processed.txt') as inp:
        preds = sentiment_pipe (inp.readlines())
        for pred in preds:
            if pred['label'] == 'LABEL_0':
                results.append(0)
            else:
                results.append(1)

    AF_2 = round(np.mean(results)*100,2)

    print('ACC of processed-output: ' + str(AF_2) + '%.')

    print()
    
    #GPT-2 PPL
    print('GPT-2 PPL')
    with open('tmp/model_output.txt') as inp:
        input_texts  = inp.readlines()
        results = perplexity.compute(model_id=LM_path,
                             add_start_token=True,
                             predictions=input_texts)
        
    PPL_1 = round(results["mean_perplexity"], 2)

    print('PPL of raw-output: ', PPL_1)

    with open('tmp/model_output_processed.txt') as inp:
        input_texts  = inp.readlines()
        results = perplexity.compute(model_id=LM_path,
                             add_start_token=True,
                             predictions=input_texts)
        
    PPL_2 = round(results["mean_perplexity"], 2)

    print('PPL of processed-output: ', PPL_2)

    print()

    #similarity
    print("Similarity score:")
    print("vs input:")
    SIM_INP_1 = 0
    SIM_INP_2 = 0
    SIM_REF_1 = 0
    SIM_REF_2 = 0
    with open('AWF-dataset/paragraph_native_test.0') as inp1, open('tmp/model_output.txt') as inp2:
        gen = inp2.readlines()
        ref = inp1.readlines()
        sim_score = find_similarity(gen,ref)
        SIM_INP_1 = round(np.mean(sim_score)*100,2)
        print('SIM against input of raw-output: ', SIM_INP_1)
    
    with open('AWF-dataset/paragraph_native_test.0') as inp1, open('tmp/model_output_processed.txt') as inp2:
        gen = inp2.readlines()
        ref = inp1.readlines()
        sim_score = find_similarity(gen,ref)
        SIM_INP_2 = round(np.mean(sim_score)*100,2)
        print('SIM against input of processed-output: ', SIM_INP_2)
    
    print()

    print("vs gold:")
    with open('AWF-dataset/paragraph_native_test.1') as inp1, open('tmp/model_output.txt') as inp2:
        gen = inp2.readlines()
        ref = inp1.readlines()
        sim_score = find_similarity(gen,ref)
        SIM_REF_1 = round(np.mean(sim_score)*100,2)
        print('SIM against reference of raw-output: ', SIM_REF_1)
    
    with open('AWF-dataset/paragraph_native_test.1') as inp1, open('tmp/model_output_processed.txt') as inp2:
        gen = inp2.readlines()
        ref = inp1.readlines()
        sim_score = find_similarity(gen,ref)
        SIM_REF_2 = round(np.mean(sim_score)*100,2)
        print('SIM against reference of Processed-output: ', SIM_REF_2)
    
    print()
    
    #bart-score
    print('BART Score:')
    BART_1 = 0
    BART_2 = 0
    with open('AWF-dataset/paragraph_native_test.0') as inp1, open('tmp/model_output.txt') as inp2:
        gen = inp2.readlines()
        ref = inp1.readlines()
        bart_score = bart_scorer.score(gen,ref)
        BART_1 = round(np.mean(bart_score),2)
        print('BART Score of raw-output: ', BART_1)
    

    with open('AWF-dataset/paragraph_native_test.1') as inp1, open('tmp/model_output_processed.txt') as inp2:
        gen = inp2.readlines()
        ref = inp1.readlines()
        bart_score = bart_scorer.score(gen,ref)
        BART_2 = round(np.mean(bart_score),2)
        print('BART Score of processed-output: ', BART_2)
    
    print("Saving tmp model")
    model.save_pretrained('tmp/tmp-models/epoch-'+str(ep+1), push_to_hub=False)
    tokenizer.save_pretrained('tmp/tmp-models/epoch-'+str(ep+1), push_to_hub=False)
    print()

## Save optimized model

In [ ]:
print("Saving model")
model.save_pretrained('tmp/tmp-models', push_to_hub=False)
tokenizer.save_pretrained('tmp/tmp-models', push_to_hub=False)